In [11]:
from selenium import webdriver
import requests
import time
from time import sleep
import csv
import bs4
from bs4 import BeautifulSoup
import pandas as pd


In [12]:
# parse html with selenium
teams = []
driver = webdriver.Chrome('/Users/jonahmiller/Downloads/chromedriver 2')
driver.get("https://barttorvik.com/trank.php?year=2021#")

for i in range(1,104):
    team_location = "/html/body/div[1]/div/table/tbody/tr[{}]".format(i)
    elem = driver.find_element_by_xpath(team_location)
    team_name = elem.get_attribute('id')
    teams.append(team_name)
    
driver.close()

scrape_team = []
names = []
for i in range(len(teams)):
    if len(teams[i]) == 0:
        continue
    names.append(teams[i][1:])
check = teams

for i in range(len(names)):
    teams[i] = names[i]
    team = teams[i]
    if len(team) == 0:
        continue
    if '_' in teams[i]:
        teams[i] = teams[i].replace('_','+')
    if team == 'Saint_Mary_s':
        teams[i] = "Saint+Mary's"
    if team == 'St__John_s':
        teams[i] = "St+John's"
        

    scrape_team.append(teams[i])
    
# Clean up the end of the names 

for i in range(len(scrape_team)):
    last = scrape_team[i][-1]
    if last == '+':
        ins = scrape_team[i][:-1] + '.'
        scrape_team[i] = ins
    if scrape_team[i] == 'St++Bonaventure':
        scrape_team[i] = 'St.+Bonaventure'

In [16]:
print(scrape_team)

['Gonzaga', 'Baylor', 'Wisconsin', 'Texas', 'Tennessee', 'Illinois', 'Houston', 'Michigan', 'Villanova', 'Iowa', 'Virginia', 'Texas+Tech', 'Creighton', 'Saint+Louis', 'Colorado', 'Rutgers', 'Oregon', 'Clemson', 'Florida', 'Indiana', 'Kansas', 'USC', 'San+Diego+St.', 'West+Virginia', 'Missouri', 'Arkansas', 'Connecticut', 'Florida+St.', 'Syracuse', 'Duke', 'Ohio+St.', 'Minnesota', 'North+Carolina+St.', 'Seton+Hall', 'Mississippi', 'LSU', 'Arizona', 'Louisville', 'Alabama', 'Oklahoma', 'Penn+St.', 'UCLA', 'Purdue', 'Stanford', 'Xavier', 'Loyola+Chicago', 'VCU', 'Northwestern', 'Maryland', 'Oklahoma+St.', 'Drake', 'North+Carolina', 'Marquette', 'Providence', 'BYU', 'Virginia+Tech', 'Boise+St.', 'Michigan+St.', 'Kentucky', 'Arizona+St.', "Saint+Mary's", 'Richmond', 'Memphis', 'Georgia+Tech', 'TCU', 'DePaul', 'St.+Bonaventure', 'SMU', 'Davidson', 'Rhode+Island', 'Utah+St.', 'Miami+FL', 'Wichita+St.', 'Utah', 'Mississippi+St.', 'Notre+Dame', 'Toledo', 'UAB', 'North+Texas', 'Pittsburgh', 'Fur

In [326]:
# driver = webdriver.Chrome('/Users/jonahmiller/Downloads/chromedriver 2')
# for i in range(10):
#     vals = []
#     link = "https://barttorvik.com/team.php?team={}&year=2021".format(scrape_team[i])
#     driver.get(link)
#     vals.append(teams[i])
#     record = '/html/body/div/div/div[2]/div[4]/table/tbody/tr[1]'
#     driver.find_element_by_xpath(record)
#     rec = elem.get_attribute('textContent')
    
# driver.close()

In [13]:
# /html/body/div/div/div[2]/div[4]/table/tbody/tr[2]


# Here is where I do the bulk of the work.  Using the scraped list of top 100 teams I navigate to their respective 
# pages which contain 2020 - 2021 season with respective records and game stats.
# As the season continues this will continue to get larger and larger
# It would make sense to have a system set up to link this data to sql database and then can effeciently manipulate data


# Future idea: Have a live feed of mathchups for the day and then automatically view algo odds 


vals = []
driver = webdriver.Chrome('/Users/jonahmiller/Downloads/chromedriver 2')
stats = []
total_team_data = {}
for i in range(len(scrape_team)):
    stats = []
    link = "https://barttorvik.com/team.php?team={}&year=2021".format(scrape_team[i])
    driver.get(link)
    vals.append(teams[i])
    record = '/html/body/div/div/h1/span'
    driver.find_element_by_xpath(record)
    for k in range(1,13):
        vals = [scrape_team[i]]
        for j in range(2,31):
            path = "/html/body/div/div/div[2]/div[4]/table/tbody/tr[{}]/td[{}]".format(k,j)
            elem = driver.find_element_by_xpath(path)
            elem = elem.get_attribute('textContent')
            vals.append(elem)
        stats.append(vals)
    total_team_data[scrape_team[i]] = stats
driver.close()

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=87.0.4280.88)


In [51]:
# print(len(total_team_data))

# Vals that we care about --> Result, 
Michigan_State = total_team_data["Michigan+St."]
Rutgers = total_team_data['Rutgers']

df_Michigan = pd.DataFrame(Michigan_State)  
df_Rutgers = pd.DataFrame(Rutgers)

df_Michigan[12] = df_Michigan[12].astype=float


# df_Michigan.drop([4,10,30], axis=1)
# df_Rutgers.drop([4,10,30], axis=1)

# df_Michigan.columns = Cols


TypeError: unsupported operand type(s) for +: 'type' and 'type'

In [37]:
# Here is a list of all the actual column names from the site.
# Need to actually walk through the scraping results to see what is pulled,
# There are most likely some inconsistencies. 

Cols = "'Team', 'Court','Unknown', ‘Opponent’, 'Opponent Abbreviation', ‘Result’, 'Tempo', ‘Record’, ‘WAB’, ‘ADJO’, “ADJD’, ‘O - PPP’, ‘O - EFG%’, ‘O - TO%’, ‘O - OR%’, ‘O - FTR’, ‘O - 2P’, ‘O - 3P’,  ‘D - PPP’, ‘D - EFG%’, ‘D - TO%’, ‘D - OR%’, ‘D -  FTR’, ‘D -  2P’, ‘D - 3P’, ‘G-SC’, ‘Average Lead/Deficit‘"
Cols = Cols.replace("‘", "")
Cols = Cols.replace('"', "")
Cols = Cols.split(',')
print(Cols)

["'Team'", " 'Court'", "'Unknown'", ' Opponent’', " 'Opponent Abbreviation'", ' Result’', " 'Tempo'", ' Record’', ' WAB’', ' ADJO’', ' “ADJD’', ' O - PPP’', ' O - EFG%’', ' O - TO%’', ' O - OR%’', ' O - FTR’', ' O - 2P’', ' O - 3P’', '  D - PPP’', ' D - EFG%’', ' D - TO%’', ' D - OR%’', ' D -  FTR’', ' D -  2P’', ' D - 3P’', ' G-SC’', ' Average Lead/Deficit']


In [10]:
df = pd.read_csv('total_team_data.csv')
df['Gonzaga']

0     ['Gonzaga', '2', '23 (I)', ' ', 'Kansas', 'KU'...
1     ['Gonzaga', '1', '87 (III)', ' ', 'Auburn', 'A...
2     ['Gonzaga', '1', '18 (I-A)', ' ', 'West Virgin...
3     ['Gonzaga', '3', '8 (I-A)', ' ', 'Iowa', 'Iowa...
4     ['Gonzaga', '2', '283 (IV)', ' ', 'Northwester...
5     ['Gonzaga', '2', '283 (IV)', ' ', 'Northwester...
6     ['Gonzaga', '2', '10 (I-A)', ' ', 'Virginia', ...
7     ['Gonzaga', '2', '324 (IV)', ' ', 'Northern Ar...
8     ['Gonzaga', '2', '329 (IV)', ' ', 'Dixie St.',...
9     ['Gonzaga', '2', '83 (III)', '• ', 'San Franci...
10    ['Gonzaga', '', '130 (III)', '• ', 'Santa Clar...
11    ['Gonzaga', '', '278 (IV)', '• ', 'Portland', ...
Name: Gonzaga, dtype: object

In [9]:
df = pd.DataFrame(total_team_data)  
df.to_csv('total_team_data.csv')

In [ ]:
for team in scrape_team:
    df = pd.DataFrame(total_team_data[team])
    name = '{}-stats.csv'.format(team)
    df.to_csv()

In [66]:
import statsmodels.api as sm
import numpy as np
# from statsmodels.api import logit
r = np.ones(20)
r = [v += 3 for v in r]
n = np.array([[3],[4.5]])

X, y = logit_data(n[-1], p, theta)

model = sm.Logit(y, X)
result = model.fit()

print(result.summary())

SyntaxError: invalid syntax (<ipython-input-66-cd5f3624c823>, line 5)